- 학습이 완료된 모델을 통해서 각 입력특성의 가중치를 확인해보자

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [2]:
house_data = pd.read_csv('data/melb_data.csv')
house_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [3]:
house_data.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [25]:
feature_names = ['Distance','Rooms','Bedroom2','Bathroom','Landsize','Lattitude','Longtitude']
X = house_data[feature_names]
y= house_data['Price']

In [26]:
# 훈련데이터와 평가데이터로 분리
from sklearn.model_selection import train_test_split

In [27]:
X_train,X_test,y_train,y_test = train_test_split(X,y,
                                                random_state=217)

In [28]:
X_train.shape, y_train.shape

((10185, 7), (10185,))

In [29]:
X_test.shape, y_test.shape

((3395, 7), (3395,))

#### 선형모델
- LinearRegression: 전통적인 수학적 해석방법으로 학습하는 모델

In [12]:
from sklearn.linear_model import LinearRegression

In [30]:
linear_model = LinearRegression()

In [31]:
linear_model.fit(X_train, y_train)

LinearRegression()

#### 예측 및 평가

In [15]:
from sklearn.metrics import mean_absolute_error

In [32]:
pre = linear_model.predict(X_test)

In [33]:
mean_absolute_error(y_test,pre)   # 회귀모델에서의 오차

319616.00328091194

#### 선형모델 수식 및 가중치 확인

- 'Distance','Rooms','Bedroom2','Bathroom','Landsize','Lattitude','Longtitude'

y = (-4.3029*Distance) + (218247*Rooms) + (14003*Bedroom2) + (216277*Bathroom) + (4*Landsize) + (-1368718*Lattitude) + (444414*Longtitude) - 116131066 
  - 도시로부터의 위치(Distance), 위도(Lattitude),경도(Longtitude)값이 영향을 많이 미치네?

In [34]:
linear_model.coef_  # 입력특성의 가중치

array([-4.30294977e+04,  2.66180458e+05,  5.60162325e+04,  1.80449614e+05,
        1.66647576e+01, -1.61995815e+06,  9.01061420e+05])

In [24]:
linear_model.intercept_

-116131066.65947142

### Xgboost사용하기

- tree 기반 boosting
 - max_death
 - random-state
 - gamma(min_samples_split): 작으면 모델이 복잡, 커비면 모델이 단순

In [37]:
!pip install xgboost

In [38]:
from xgboost import XGBRegressor

- booster: 부스팅할 모델 선택
    - gbtree, gblinear
- objective : 부스팅을 할 때 모델의 비용함수 설정
    - 이진분류-> binary:logistic
    - 다중분류-> multi:softmax
    - 회귀 -> reg:squarederror
- n_estimators : 부스팅으로 몇 번이나 모델을 개선할건지 설정

In [52]:
xgbr_model = XGBRegressor(booster = 'gbtree',
                         objective='reg:squarederror',
                         n_estimators =5000,
                         random_state=217)

In [53]:
xgbr_model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=5000, n_jobs=12, num_parallel_tree=1,
             random_state=217, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [54]:
xgbr_pre= xgbr_model.predict(X_test)

In [55]:
mean_absolute_error(y_test,xgbr_pre)

199136.6467715908